In [1]:
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account, get_local_accounts

In [2]:
# initialise the client
# client = SpeckleClient(host="your-server.com") # or whatever your host is
client = SpeckleClient(host="speckle.xyz", use_ssl=True)

# authenticate the client with a token
account = get_default_account()
client.authenticate(token=account.token)

In [3]:
# create a new stream. this returns the stream id
new_stream_id = client.stream.create(name="earthy_test_2")

In [4]:
# use that stream id to get the stream from the server
new_stream = client.stream.get(id=new_stream_id)

In [12]:
from specklepy.objects import Base
from specklepy.objects.geometry import Point, Box, Vector, Plane, Interval

class Block(Base):
    length: float = 1.0
    width: float = 1.0
    height: float = 1.0
    origin: Point = Point()

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        # mark the origin as a detachable attribute
        self.add_detachable_attrs("origin")

In [24]:
from specklepy.transports.server import ServerTransport
# from specklepy.transports.memory import MemoryTransport
# from specklepy.transports.sqlite import SQLiteTransport
from specklepy.api import operations

# here's the data you want to send
# block = Block(length=2, height=22)
origin_point = Point(x=0, y=0, z=0)
x_dir = Vector(x=1, y=0, z=0)
y_dir = Vector(x=0, y=1, z=0)
z_dir = Vector(x=0, y=0, z=1)
base_plane = Plane(origin=origin_point, normal=z_dir, xdir=x_dir, ydir=y_dir)
x_interval = Interval(start=0, end=1)
y_interval = Interval(start=0, end=3)
z_interval = Interval(start=0, end=1)
mybox = Box(basePlane=base_plane, xSize=x_interval, ySize=y_interval, zSize=z_interval)
# block = Block(length=2, height=22)

In [28]:


# next create a server transport - this is the vehicle through which you will send and receive
transport = ServerTransport(client=client, stream_id=new_stream_id)

# this serialises the block and sends it to the transport
hash = operations.send(base=mybox, transports=[transport])

# you can now create a commit on your stream with this object
commid_id = client.commit.create(
    stream_id=new_stream_id, 
    object_id=hash, 
    message="new box",
    )

In [49]:
# this receives the object back from the transport.
# the received data will be deserialised back into a `Block` 
received_base = operations.receive(obj_id=hash, remote_transport=transport)
received_base.height

12.0

In [50]:
# get list of commits
client.commit.list(new_stream_id)

[Commit( id: 2f4644fb4f, message: this is a block I made in speckle-py, referencedObject: 7c79b716f43c12dcea442c10e1c5c027, authorName: Shervin Azadi, createdAt: 2021-07-15T15:58:12.199Z ),
 Commit( id: 5b418686ee, message: this is a block I made in speckle-py, referencedObject: 4250fb1b7fd2d368662655036c7b23ec, authorName: Shervin Azadi, createdAt: 2021-07-15T15:56:42.381Z )]

In [9]:
import specklepy.objects.geometry as sog


In [ ]:
sog.